In [2]:
from distutils.log import info
import datetime
import pandas as pd
import numpy as np
from  SQL_e import sql_datenLadenLabel,sql_datenLadenOderItems,sql_datenLadenStammdaten,sql_datenLadenOder,createnewTable

def stammdatenBearbeiten():
    dfStammdaten = sql_datenLadenStammdaten()
    dfStammdaten = dfStammdaten[dfStammdaten['UnitOfMeasure'].isin(['CS','D97','OUT'])]   
    dfStammdaten['MaterialNumber'] = dfStammdaten['MaterialNumber'].str.replace('0000000000', '')
    
    def f_CS(row):
        try:
            if row.UnitOfMeasure == 'CS':          
                return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
        except:
            return np.nan
    def f_PAL(row):
        try:
            if row.UnitOfMeasure == 'D97':
                return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
        except:
            return np.nan
    def f_OUT(row):
        try:
            if row.UnitOfMeasure == 'OUT':
                return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
        except:
            return np.nan
    dfStammdaten['OUT'] = dfStammdaten.apply(f_OUT,axis=1)
    dfStammdaten['CS'] = dfStammdaten.apply(f_CS,axis=1)
    dfStammdaten['PAL'] = dfStammdaten.apply(f_PAL,axis=1)

    return dfStammdaten



In [3]:
dfOrderItems = sql_datenLadenOderItems()

In [4]:

dfOrder = sql_datenLadenOder()

In [5]:
dfStammdaten = stammdatenBearbeiten()

In [6]:
###TODO Die zusammenfassung der Items in der Datenbank verfälschen die Anzahl Picks PAL und CS
### da die Positionen einzeln gepickt werden sind sie nicht als BS3 Pick navigiert und damit kein Pal Pick 

dfOrderItems['MaterialNumber'] = dfOrderItems['MaterialNumber'].astype(str)
dfOrderItems['MaterialNumber'] = dfOrderItems['MaterialNumber'].str.replace('0000000000', '')
dfOrderItems = pd.merge(dfOrderItems, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'CS'][['MaterialNumber','CS']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')
dfOrderItems = pd.merge(dfOrderItems, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'D97'][['MaterialNumber','PAL']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')
dfOrderItems = pd.merge(dfOrderItems, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'OUT'][['MaterialNumber','OUT']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')

dfOrderItems['O'] = dfOrderItems['Outers'] * dfOrderItems['OUT']
dfOrderItems['PicksPAL'] = dfOrderItems.O / dfOrderItems.PAL
dfOrderItems['PicksCS'] = dfOrderItems.CorrespondingCartons 
dfOrderItems['Picks OUT'] = dfOrderItems.CorrespondingOuters
#Bereinige Berechnungen der Picks 
dfOrderItems['PicksPAL'] = dfOrderItems['PicksPAL'].fillna(0) 
dfOrderItems['O'] = dfOrderItems.O.fillna(0) 
dfOrderItems['Picks OUT'] = dfOrderItems['Picks OUT'].fillna(0) 

# Pal kleiner als 1 bereinigen
def f_PAL(row):
        if row.PicksPAL < 1:
            return 0
        else:
            return row.PicksPAL

dfOrderItems['Picks PAL'] = dfOrderItems.apply(f_PAL,axis=1)

def f_OUT(row):
        if row.PicksPAL >= 1:
            return row.CorrespondingCartons  - ((row.PicksPAL * row.PAL) / row.CS)
        else:
            return row.CorrespondingCartons

dfOrderItems['Picks CS'] = dfOrderItems.apply(f_OUT,axis=1)

dfOrderItems['Picks Gesamt'] = dfOrderItems['Picks PAL'] + dfOrderItems['Picks CS'] + dfOrderItems['Picks OUT']
df = pd.merge(dfOrder, dfOrderItems, left_on='SapOrderNumber', right_on='SapOrderNumber', how='left')

# sap order 3202209324
# h = dfOrderItems[dfOrderItems['SapOrderNumber'] == '3202209324']
# h

In [7]:
dfLabel = sql_datenLadenLabel()
dfLabel


,ID,CreatedTimestamp,CreatedBy,UpdatedTimestamp,UpdatedBy,SapOrderNumber,SSCC,Printer,Copies,Status,ErrorText,DangerousGoodsUNNumber,DangerousGoodsWeight,LabelTemplate
0,1,2022-09-03 09:29:47.351169+00:00,Patrick Reuschel,None,None,3202058418,340313000104427053,DEBY-P475,1,Printed,None,None,NaN,../Labels/SuperDepot_90_40.nlbl
1,2,2022-09-03 09:34:13.027480+00:00,85125888,None,None,3202058418,340313000104427053,DEBY-P475,1,Printed,None,None,NaN,../Labels/SuperDepot_148_210.nlbl
2,3,2022-09-03 09:37:44.669025+00:00,Bernd Schwing,None,None,3202058417,659031410522929565,DEBY-P471,1,Printed,None,None,NaN,../Labels/SuperDepot_148_210.nlbl
3,4,2022-09-03 09:40:48.573843+00:00,Bernd Schwing,None,None,3202058417,140313001112609048,DEBY-P471,1,Printed,None,None,NaN,../Labels/SuperDepot_148_210.nlbl
4,5,2022-09-03 09:40:55.699888+00:00,Patrick Reuschel,None,None,3202058419,340313000104464560,DEBY-P473,1,Printed,None,None,NaN,../Labels/SuperDepot_148_210.nlbl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14362,14363,2022-12-02 18:01:31.605086+00:00,Susanne Niporschke,None,None,3202256060,340313000110394219,DEBY-P556,1,Printed,None,None,NaN,../Labels/SuperDepot_103_150.nlbl
14363,14364,2022-12-02 18:01:44.309974+00:00,Susanne Niporschke,None,None,3202256060,340313000110394202,DEBY-P556,1,Printed,None,None,NaN,../Labels/SuperDepot_103_150.nlbl
14364,14365,2022-12-02 18:01:56.355287+00:00,Susanne Niporschke,None,None,3202256060,340313000104522116,DEBY-P556,1,Printed,None,None,NaN,../Labels/SuperDepot_103_150.nlbl
14365,14366,2022-12-02 18:02:06.732027+00:00,Susanne Niporschke,None,None,3202256060,340313000104522321,DEBY-P556,1,Printed,None,None,NaN,../Labels/SuperDepot_103_150.nlbl


In [8]:
#     def labelAgg():
#         #TODO: Label aus DB laden übernehmen
#         # gibt nur Label aus die von angelegten Mitarbeitern bearbeitet wurden
#         dfUser = pd.read_feather('/Users/martinwolf/Python/Superdepot Reporting/data/user.feather')
#         dfLabel = datenLadenLabel()
        
        
#         # #dfLabel = pd.read_csv('data/Label.csv')
#         # dfLabel.columns = ['A', 'B','C','D','E','F','G','H','I','J','K','L','M','N']
        
#         #dfLabel['CreatedTimestamp'] = pd.to_datetime(dfLabel['CreatedTimestamp'])
#         # dfLabel['DATUM'] = dfLabel['CreatedTimestamp'].dt.strftime('%m/%d/%y')
#         # dfLabel['TIME'] = dfLabel['CreatedTimestamp'].dt.strftime('%H:%M:%S')
#         # dfLabel['TIME'] = dfLabel['TIME'] + pd.Timedelta(hours=1)
#         # dfLabel['TIME'] = dfLabel['CreatedTimestamp'].dt.strftime('%H:%M:%S')
#         dfLabel = pd.merge(dfLabel, dfUser, left_on='CreatedBy', right_on='Name')
#         return dfLabel